In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# =========================
# 1. Global configuration
# =========================

# Price Grid
price_levels = np.array([x/100 for x in range(1,100)])
n_prices = len(price_levels)

# Horizon: number of trading steps
T = 20

In [3]:
# =====================================
# 2. Model utilities and DP primitives
# =====================================

def make_transition_matrix(p_up: float, p_down: float) -> np.ndarray:
    """
    Build a Markov transition matrix over the discrete price grid.
    p_up, p_down: probabilities of moving up/down one grid level.
    Remaining mass is p_same. Probabilities are renormalized at edges.
    """
    assert (p_up + p_down) <= 1, "Probabilities must sum to 1"
    p_same = 1.0 - p_up - p_down
    M = np.zeros((n_prices, n_prices))

    for i in range(n_prices):
        probs = []
        dests = []

        # down
        if i > 0:
            probs.append(p_down)
            dests.append(i - 1)

        # same
        probs.append(p_same)
        dests.append(i)

        # up
        if i < n_prices - 1:
            probs.append(p_up)
            dests.append(i + 1)

        probs = np.array(probs, dtype=float)

        for p, j in zip(probs, dests):
            M[i, j] += p

    return M